In [ ]:
# List all NVIDIA GPUs as avaialble in this computer (or Colab's session)
!nvidia-smi -L

GPU 0: A100-SXM4-40GB (UUID: GPU-a40c3943-09aa-5990-efd1-2b842c15e15c)


In [ ]:
import sys
print( f"Python {sys.version}\n" )

import numpy as np
print( f"NumPy {np.__version__}\n" )

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

Python 3.7.14 (default, Sep  8 2022, 00:06:44) 
[GCC 7.5.0]

NumPy 1.21.6

TensorFlow 2.9.2
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 1 GPU(s):
.... GPU No. 0: Name = /physical_device:GPU:0 , Type = GPU


In [ ]:
# Load the ImageNet VGG-16 model, including both the feature extractor part and the classifier part
# By default, VGG-16 is designed for an input image of dimension 224x224x3
vgg = tf.keras.applications.vgg16.VGG16(weights = "imagenet", include_top=True)

vgg.summary()

553467096/553467096 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [ ]:
def load_images_from_folder(folder,only_path = False, label = ""):
    if only_path == False:
        images = []
        for filename in os.listdir(folder):
            img = plt.imread(os.path.join(folder,filename))
            if img is not None:
                images.append(img)
        return images
    else:
        path = []
        for filename in os.listdir(folder):
            img_path = os.path.join(folder,filename)
            if img_path is not None:
                path.append([label,img_path])
        return path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
https://drive.google.com/drive/folders/1EqX9QAHqX8TL6a0LGSDvL0Uks0jngQ_9?usp=sharing

In [ ]:
import os
import pandas as pd

images = []
dirp = "/content/drive/MyDrive/Colab Notebooks/DADS7202_DeepLearning/01_DataSet_image/"
for f in os.listdir(dirp):
    if "jpg" in os.listdir(dirp+f)[0]:
        images += load_images_from_folder(dirp+f,True,label = f)
    else: 
        for d in os.listdir(dirp+f):
            images += load_images_from_folder(dirp+f+"/"+d,True,label = f)
            
df = pd.DataFrame(images, columns = ["fruit", "path"])

from sklearn.utils import shuffle
df = shuffle(df, random_state = 0)
df = df.reset_index(drop=True)

fruit_names = sorted(df.fruit.unique())
mapper_fruit_names = dict(zip(fruit_names, [t for t in range(len(fruit_names))]))
df["label"] = df["fruit"].map(mapper_fruit_names)
print(mapper_fruit_names)

df

{'Long_Eggplant': 0, 'Pea_Eggplant': 1, 'Thai_Eggplant': 2, 'Tomato': 3}


,fruit,path,label
0,Long_Eggplant,/content/drive/MyDrive/Colab Notebooks/DADS720...,0
1,Long_Eggplant,/content/drive/MyDrive/Colab Notebooks/DADS720...,0
2,Thai_Eggplant,/content/drive/MyDrive/Colab Notebooks/DADS720...,2
3,Long_Eggplant,/content/drive/MyDrive/Colab Notebooks/DADS720...,0
4,Pea_Eggplant,/content/drive/MyDrive/Colab Notebooks/DADS720...,1
...,...,...,...
623,Long_Eggplant,/content/drive/MyDrive/Colab Notebooks/DADS720...,0
624,Tomato,/content/drive/MyDrive/Colab Notebooks/DADS720...,3
625,Pea_Eggplant,/content/drive/MyDrive/Colab Notebooks/DADS720...,1
626,Long_Eggplant,/content/drive/MyDrive/Colab Notebooks/DADS720...,0


In [ ]:
img =  tf.keras.preprocessing.image.load_img(df.path[0:3], target_size=(224, 224, 3))   
print( f"type(img) = {type(img)}" )

# Convert PIL Image to NumPy ndarray
x = tf.keras.preprocessing.image.img_to_array(img) 
print( f"type(x) = {type(x)} , x.dtype = {x.dtype} , x.shape = {x.shape} , min(x) = {np.min(x)} , max(x) = {np.max(x)}" )

# Display the NumPy image
plt.imshow( x.astype(np.uint8) )
plt.show()


TypeError: ignored

In [ ]:
# Preprocess the image following requirements of VGG-16
x = tf.keras.applications.vgg16.preprocess_input(x)

print( f"type(x) = {type(x)} , x.dtype = {x.dtype} , x.shape = {x.shape} , min(x) = {np.min(x):.4f} , max(x) = {np.max(x):.4f}" )

type(x) = <class 'numpy.ndarray'> , x.dtype = float32 , x.shape = (224, 224, 3) , min(x) = -123.6800 , max(x) = 151.0610


In [ ]:
# Use the model to predict this image
pred = vgg.predict( np.expand_dims(x, axis=0) )

# Decode the top 10 classes
predictions = tf.keras.applications.vgg16.decode_predictions(pred, top=10)[0]
for i, prediction in enumerate(predictions):
  print( f"Rank {i+1}: {prediction}" )

35363/35363 [==============================] - 0s 0us/step
Rank 1: ('n01644373', 'tree_frog', 0.41564578)
Rank 2: ('n02236044', 'mantis', 0.3282092)
Rank 3: ('n01682714', 'American_chameleon', 0.12305075)
Rank 4: ('n01693334', 'green_lizard', 0.098862365)
Rank 5: ('n02231487', 'walking_stick', 0.004527805)
Rank 6: ('n01677366', 'common_iguana', 0.004356582)
Rank 7: ('n07718472', 'cucumber', 0.004190755)
Rank 8: ('n01749939', 'green_mamba', 0.0021949227)
Rank 9: ('n01694178', 'African_chameleon', 0.0020358828)
Rank 10: ('n02226429', 'grasshopper', 0.0018825876)
